<a href="https://colab.research.google.com/github/vcorredorg/SenalesySistemas2025/blob/main/Parcial2/Parcial2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div align="center">

### **PARCIAL #2 SEÑALES Y SISTEMAS - 2025 2S**
#### **Estudiante:** Valeria Corredor García
##### Departamento de Ingeniería Eléctrica, Electrónica y Computación
##### Universidad Nacional de Colombia - Sede Manizales

</div>

---

##**Instalación de librerías**



In [1]:
!pip install streamlit yt-dlp librosa soundfile

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.0/180.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 30.3 MB/s eta 0:00:00


In [2]:
!mkdir -p pages

##**Página de inicio**

In [3]:
%%writefile app.py
import streamlit as st

st.set_page_config(
    page_title="Parcial 2 - Señales y Sistemas",
    page_icon="📡",
    layout="wide"
)

st.title("Parcial 2 - Señales y Sistemas 📡")

st.markdown(
    """
Este dashboard interactivo corresponde al **Parcial 2** del curso de *Señales y Sistemas*.

El tema central de estudio es la implementación y análisis de:

- Transformada de Fourier
- Transformada de Laplace
- Modulación en amplitud (AM)

A continuación se presentan diferentes **secciones** (páginas) que profundizan en cada uno de estos puntos.

Usa el menú lateral de Streamlit (barra izquierda) para navegar entre las páginas.
    """
)


Writing app.py


##**Páginas**

###1.Demodulación AM (DSB-SC) con FFT

####Importar imágen del demodulador

In [4]:
from google.colab import files

uploaded = files.upload()


Saving demodulador.png to demodulador.png


####Página modulación y demodulación AM

In [5]:
%%writefile pages/1_📻_AM_Modulacion_Demodulacion.py
import streamlit as st
import numpy as np
import matplotlib.pyplot as plt
import librosa
import soundfile as sf
import yt_dlp
from yt_dlp.utils import DownloadError
import io

#Funciones auxiliares

def descargar_segmento_youtube(url, start_sec=20, duration_sec=5, target_sr=44100, cookies_bytes=None):
    """
    Descarga audio de YouTube, recorta [start_sec, start_sec+duration_sec]
    y normaliza. Si cookies_bytes no es None, se usan como cookies.txt.
    """
    ydl_opts = {
        "format": "bestaudio/best",
        "noplaylist": True,
        "quiet": True,
        "outtmpl": "yt_audio.%(ext)s",
    }

    # Configurar cookies
    if cookies_bytes is not None:
        cookiefile = "cookies_youtube.txt"
        with open(cookiefile, "wb") as f:
            f.write(cookies_bytes)
        ydl_opts["cookiefile"] = cookiefile

    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info = ydl.extract_info(url, download=True)
            filename = ydl.prepare_filename(info)
    except DownloadError as e:
        raise RuntimeError(
            "No se pudo descargar el audio desde YouTube.\n\n"
            "Prueba lo siguiente:\n"
            "- Si ya subiste cookies.txt, revisa que sean del dominio youtube.com.\n"
        ) from e

    # Cargar audio como mono
    audio, sr = librosa.load(filename, sr=target_sr, mono=True)

    start = int(start_sec * sr)
    end = min(start + int(duration_sec * sr), len(audio))
    segmento = audio[start:end]

    # Normalización
    max_val = np.max(np.abs(segmento))
    if max_val > 0:
        segmento = segmento / max_val

    return segmento, sr


def a_bytes_wav(x, sr):
    buf = io.BytesIO()
    sf.write(buf, x, sr, format="WAV")
    buf.seek(0)
    return buf


def espectro(x, sr):
    N = len(x)
    freqs = np.fft.rfftfreq(N, d=1.0 / sr)
    X = np.fft.rfft(x)
    mag = np.abs(X) / N
    return freqs, mag


def lowpass_fft(x, sr, cutoff_hz):
    N = len(x)
    X = np.fft.rfft(x)
    freqs = np.fft.rfftfreq(N, d=1.0 / sr)
    H = (freqs <= cutoff_hz).astype(float)
    Y = X * H
    y = np.fft.irfft(Y, n=N)
    return y


def graficar_tiempo_y_frecuencia(x, sr, titulo, max_freq=None):
    t = np.arange(len(x)) / sr
    freqs, mag = espectro(x, sr)

    fig, axs = plt.subplots(1, 2, figsize=(12, 4))

    axs[0].plot(t, x)
    axs[0].set_xlabel("Tiempo [s]")
    axs[0].set_ylabel("Amplitud")
    axs[0].set_title(titulo + " (tiempo)")

    axs[1].plot(freqs, mag)
    axs[1].set_xlabel("Frecuencia [Hz]")
    axs[1].set_ylabel("|X(f)|")
    axs[1].set_xlim(0, max_freq if max_freq is not None else sr / 2)
    axs[1].set_title(titulo + " (espectro)")

    st.pyplot(fig)
    st.audio(a_bytes_wav(x, sr), format="audio/wav")


#Configuración de la página
st.set_page_config(
    page_title="Modulación y Demodulación AM",
    page_icon="📻",
    layout="wide"
)

st.title("Modulación y demodulación AM usando audio de YouTube")

st.markdown(
    """
Sea el demodulador en amplitud presentado en la siguiente
 Figura:""")

st.image("demodulador.png", caption="Diagrama del demodulador AM", use_container_width=True)

st.markdown(
    """
Asumiendo θ0 = 0, utilizando cinco segundos de una canción de Youtube como mensaje, grafique cada una de las etapas principales del proceso de modulación y demodulaciónn en el tiempo y la frecuencia (reproduzca el segmento de la canción en cada etapa). Para la etapa de filtrado pasa bajas, realizar la implementación a partir de la transformada rápida de Fourier.

Sea la señal portadora:
$$c(t) = A_c \\cos(2\\pi f_c t)$$

y una señal mensaje m(t). Para demodular, se usa un mezclador y un filtro pasa bajas implementado con FFT.
"""
)

#Controles de usuario
url = st.text_input("URL de YouTube", "")

fc = st.sidebar.slider(
    "Frecuencia de la portadora $f_c$ [Hz]",
    min_value=1000,
    max_value=15000,
    value=8000,
    step=500,
)

mu = st.sidebar.slider(
    "Índice de modulación μ",
    min_value=0.1,
    max_value=0.9,
    value=0.7,
    step=0.1,
)

cutoff = st.sidebar.slider(
    "Frecuencia de corte del filtro pasa bajas [Hz]",
    min_value=1000,
    max_value=8000,
    value=4000,
    step=500,
)

cookies_file = st.sidebar.file_uploader(
    "cookies.txt de YouTube (opcional pero recomendado)",
    type=["txt"]
)

st.sidebar.markdown(
    """
- Se toman 5 s de audio entre los segundos 20 y 25.
- El filtro pasa bajas se implementa con FFT.
"""
)

#Lógica principal
if st.button("Descargar audio y ejecutar proceso de modulación/demodulación"):
    if not url:
        st.warning("Ingresa primero una URL de YouTube.")
    else:
        cookies_bytes = cookies_file.getvalue() if cookies_file is not None else None
        with st.spinner("Descargando audio y generando señales..."):
            try:
                m, sr = descargar_segmento_youtube(
                    url,
                    start_sec=20,
                    duration_sec=5,
                    cookies_bytes=cookies_bytes
                )
            except RuntimeError as e:
                st.error(str(e))
            else:
                st.session_state["m"] = m
                st.session_state["sr"] = sr
                st.success("Audio descargado. Desplázate hacia abajo para ver las etapas.")

#Etapas de modulación y demodulación
if "m" in st.session_state:
    m = st.session_state["m"]
    sr = st.session_state["sr"]

    N = len(m)
    t = np.arange(N) / sr
    Ac = 1.0
    A1 = 1.0

    # Normalización matemática
    max_m = np.max(np.abs(m)) if np.max(np.abs(m)) > 0 else 1.0
    m_norm = m / max_m

    # 1. Mensaje
    st.subheader("1. Señal mensaje $m(t)$")
    graficar_tiempo_y_frecuencia(m, sr, "Mensaje m(t)", max_freq=8000)

    # 2. AM con portadora
    carrier = Ac * np.cos(2 * np.pi * fc * t)
    y_am = (1.0 + mu * m_norm) * carrier

    st.subheader("2. Señal AM con portadora $y(t)$")
    graficar_tiempo_y_frecuencia(y_am, sr, "Señal AM con portadora", max_freq=sr / 2)

    # 3. DSB-SC recibida
    s_dsb = A1 * m_norm * np.cos(2 * np.pi * fc * t)

    st.subheader("3. Señal DSB-SC recibida $A_1 m(t)\\cos(2\\pi f_0 t)$")
    graficar_tiempo_y_frecuencia(s_dsb, sr, "DSB-SC", max_freq=sr / 2)

    # 4. Salida del mezclador
    mixer_out = s_dsb * np.cos(2 * np.pi * fc * t)

    st.subheader("4. Salida del mezclador")
    graficar_tiempo_y_frecuencia(mixer_out, sr, "Salida del mezclador", max_freq=sr / 2)

    # 5. Filtro pasa bajas (FFT)
    lp_out = lowpass_fft(mixer_out, sr, cutoff)

    st.subheader("5. Salida del filtro pasa bajas (FFT)")
    graficar_tiempo_y_frecuencia(lp_out, sr, "Después del filtro pasa bajas", max_freq=8000)

    # 6. Señal recuperada
    recovered = (2.0 / A1) * lp_out

    st.subheader("6. Señal recuperada")
    graficar_tiempo_y_frecuencia(recovered, sr, "Mensaje recuperado", max_freq=8000)

else:
    st.info("Ingresa una URL, opcionalmente sube cookies.txt, y pulsa el botón para correr el experimento.")


Writing pages/1_📻_AM_Modulacion_Demodulacion.py


##Punto 2: Masa–resorte–amortiguador y circuito RLC

In [6]:
from google.colab import files

uploaded = files.upload()

Saving sistema_masa_resorte.png to sistema_masa_resorte.png


In [9]:
%%writefile pages/2_⚙️_Masa_Resorte_Amortiguador.py
import streamlit as st
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as sig

#Funciones auxiliares

def compute_second_order_params(m, c, k):
    """
    Calcula parámetros dinámicos de un sistema de segundo orden:
    m s^2 + c s + k, visto como s^2 + 2 ζ ω_n s + ω_n^2.
    """
    wn = np.sqrt(k/m)
    zeta = c / (2*np.sqrt(k*m))
    params = {"wn": wn, "zeta": zeta, "c": c}

    den = [m, c, k]
    poles = np.roots(den)
    params["poles"] = poles

    eps = 1e-3
    if zeta < 1 - eps:
        # Subamortiguado
        wd = wn * np.sqrt(1 - zeta**2)
        tp = np.pi / wd
        phi = np.arctan(np.sqrt(1 - zeta**2) / zeta) if zeta > 0 else np.pi/2
        tr = (np.pi - phi) / wd
        ts = 4 / (zeta * wn) if zeta > 0 else np.nan
    elif abs(zeta - 1) <= eps:
        # Crítico
        wd = None
        tp = None
        tr = None
        ts = 4 / (zeta * wn)
    else:
        # Sobreamortiguado
        wd = None
        tp = None
        tr = None
        # polo dominante (más cercano al eje imaginario)
        s_dom = poles[np.argmax(np.real(poles))]
        alpha = -np.real(s_dom)
        ts = 4 / alpha if alpha > 0 else np.nan

    params.update({"wd": wd, "tp": tp, "tr": tr, "ts": ts})
    return params


def plot_pzmap(sys, title="Diagrama de polos y ceros"):
    # Obtenemos ceros y polos desde la TF
    num, den = sys.num, sys.den
    zeros, poles, _ = sig.tf2zpk(num, den)

    fig, ax = plt.subplots(figsize=(5, 5))

    # Plano cartesiano con ejes centrados
    ax.spines["left"].set_position("zero")
    ax.spines["bottom"].set_position("zero")
    ax.spines["right"].set_color("none")
    ax.spines["top"].set_color("none")

    xs = list(poles.real)
    ys = list(poles.imag)
    if zeros is not None and len(zeros) > 0:
        xs.extend(zeros.real)
        ys.extend(zeros.imag)

    if len(xs) == 0:
        xs = [0.0]
    if len(ys) == 0:
        ys = [0.0]

    max_range = max(max(np.abs(xs)), max(np.abs(ys)), 1.0)
    R = 1.2 * max_range
    ax.set_xlim(-R, R)
    ax.set_ylim(-R, R)
    ax.set_aspect("equal", "box")

    # Polos y ceros
    if zeros is not None and len(zeros) > 0:
        ax.scatter(zeros.real, zeros.imag, marker="o", facecolors="none",
                   edgecolors="C0", label="Ceros")
    ax.scatter(poles.real, poles.imag, marker="x", color="C1", label="Polos")

    ax.set_xlabel("Re{s}")
    ax.set_ylabel("Im{s}")
    ax.set_title(title)
    ax.grid(True, which="both", linestyle=":")
    ax.legend()
    fig.tight_layout()
    return fig


def plot_bode(sys, title="Diagrama de Bode"):
    w, mag, phase = sig.bode(sys)
    fig, axs = plt.subplots(2, 1, figsize=(6, 6), sharex=True)

    axs[0].semilogx(w, mag)
    axs[0].set_ylabel("Magnitud [dB]")
    axs[0].set_title(title)
    axs[0].grid(True, which="both", ls=":")

    axs[1].semilogx(w, phase)
    axs[1].set_ylabel("Fase [deg]")
    axs[1].set_xlabel("Frecuencia [rad/s]")
    axs[1].grid(True, which="both", ls=":")

    fig.tight_layout()
    return fig


def plot_time_responses(sys, params, title="Respuestas en el tiempo"):
    wn = params["wn"]
    ts = params["ts"]

    if ts is not None and np.isfinite(ts) and ts > 0:
        t_end = 4 * ts
    elif wn > 0:
        t_end = 5 / wn
    else:
        t_end = 10.0

    t = np.linspace(0, t_end, 1000)

    fig, axs = plt.subplots(3, 1, figsize=(7, 9), sharex=True)

    # Impulso
    tout, y_imp = sig.impulse(sys, T=t)
    axs[0].plot(tout, y_imp)
    axs[0].set_title("Respuesta al impulso")
    axs[0].grid(True)

    # Escalón
    t_step, y_step = sig.step(sys, T=t)
    axs[1].plot(t_step, y_step)
    axs[1].set_title("Respuesta al escalón")
    axs[1].grid(True)

    # Rampa: u(t) = t
    u = t
    t_ramp, y_ramp, _ = sig.lsim(sys, U=u, T=t)
    axs[2].plot(t_ramp, y_ramp)
    axs[2].set_title("Respuesta a rampa")
    axs[2].grid(True)

    for ax in axs:
        ax.set_xlabel("t [s]")

    fig.suptitle(title)
    fig.tight_layout()
    return fig


def clasifica_zeta(z):
    if z < 1 - 1e-3:
        return "Subamortiguado (0 < ζ < 1)"
    elif abs(z - 1) <= 1e-3:
        return "Críticamente amortiguado (ζ ≈ 1)"
    else:
        return "Sobreamortiguado (ζ > 1)"


#Configuración de página
st.set_page_config(
    page_title="Masa–Resorte–Amortiguador",
    page_icon="⚙️",
    layout="wide",
)

st.title("Sistema masa–resorte–amortiguador: lazo abierto y lazo cerrado")

st.markdown(
    r"""
 Sea el sistema masa, resorte amortiguador, presentado en la siguiente
 Figura (con condiciones iniciales cero):""")

st.image(
    "sistema_masa_resorte.png",
    caption="Diagrama sistema masa-resorte amortiguador"
)

st.markdown(
    r"""
Con función de transferencia es:
$$
G(s)=\frac{1}{m s^{2} + c s + k}
$$

Para realizar la simulación del sistema:
1. Elige el **tipo de amortiguamiento** (sub, crítico o sobre).
2. Ajusta \(m\), \(k\) y \(c\). El valor de \(c\) se inicializa según el tipo elegido,
   pero tú lo puedes modificar.
3. Se calculan los parámetros dinámicos en **lazo abierto** y **lazo cerrado**
   (con realimentación unitaria y un controlador proporcional $$K_c$$.

Se muestran:

- $$\zeta, \omega_n, \omega_d, t_p, t_r, t_s$$.
- Diagrama de polos y ceros (plano s).
- Bode.
- Respuesta impulso, escalón y rampa.
"""
)

#Controles del usuario (barra lateral)

st.sidebar.header("Tipo de amortiguamiento a simular")
tipo = st.sidebar.radio(
    "Selecciona el régimen:",
    ("Subamortiguado", "Críticamente amortiguado", "Sobreamortiguado")
)

# zeta típico según selección, solo para proponer c por defecto
zeta_ref = {
    "Subamortiguado": 0.2,
    "Críticamente amortiguado": 1.0,
    "Sobreamortiguado": 2.0,
}[tipo]

st.sidebar.header("Parámetros físicos")

m = st.sidebar.number_input(
    "Masa m [kg]",
    min_value=0.1,
    max_value=100.0,
    value=1.0,
    step=0.1,
)

k = st.sidebar.number_input(
    "Rigidez k [N/m]",
    min_value=0.1,
    max_value=500.0,
    value=25.0,
    step=1.0,
)

wn_base = np.sqrt(k / m)
st.sidebar.write(f"Frecuencia natural no amortiguada ωₙ ≈ {wn_base:.3f} rad/s")

# c sugerido según zeta_ref
c_sugerido = 2 * zeta_ref * np.sqrt(k * m)

c = st.sidebar.number_input(
    "Amortiguamiento c [N·s/m]",
    min_value=0.0,
    max_value=1e4,
    value=float(c_sugerido),
    step=0.1,
    help="Puedes modificar c libremente; el tipo real dependerá del ζ resultante.",
)

st.sidebar.header("Control (lazo cerrado)")
Kc = st.sidebar.slider(
    "Ganancia proporcional $K_c$",
    min_value=0.0,
    max_value=200.0,
    value=25.0,
    step=1.0,
    help="Ganancia del controlador proporcional en lazo cerrado",
)

#Cálculo y visualización

st.subheader(f"Régimen seleccionado: {tipo}")

st.markdown(
    rf"""
**Parámetros físicos actuales**

- $$m = {m:.2f}\ \text{{kg}}$$
- $$k = {k:.2f}\ \text{{N/m}}$$
- $$c = {c:.2f}\ \text{{N*s/m}}$$
"""
)

#Lazo abierto

st.markdown("### Lazo abierto")

sys_ol = sig.TransferFunction([1.0], [m, c, k])
params_ol = compute_second_order_params(m, c, k)

col1, _ = st.columns([1, 1])

with col1:
    st.markdown("**Parámetros dinámicos (abierto)**")
    z_ol = params_ol["zeta"]
    wn_ol = params_ol["wn"]
    wd_ol = params_ol["wd"]
    tp_ol = params_ol["tp"]
    tr_ol = params_ol["tr"]
    ts_ol = params_ol["ts"]

    st.write(f"ζ (abierto) = {z_ol:.4f} → {clasifica_zeta(z_ol)}")
    st.write(f"ωₙ (abierto) = {wn_ol:.4f} rad/s")
    if wd_ol is not None:
        st.write(f"ωd (abierto) = {wd_ol:.4f} rad/s")
    else:
        st.write("ωd (abierto): no aplica (no hay oscilaciones).")
    st.write(f"ts (abierto) ≈ {ts_ol:.4f} s")
    if tp_ol is not None:
        st.write(f"tp (abierto) ≈ {tp_ol:.4f} s")
    else:
        st.write("tp (abierto): no aplica (respuesta monótona).")
    if tr_ol is not None:
        st.write(f"tr (abierto) ≈ {tr_ol:.4f} s")
    else:
        st.write("tr (abierto): se obtiene de la respuesta monótona.")

with st.expander("Polos y ceros (lazo abierto)"):
    fig_pz_ol = plot_pzmap(sys_ol, "Lazo abierto: polos y ceros")
    st.pyplot(fig_pz_ol)

with st.expander("Bode lazo abierto"):
    fig_bode_ol = plot_bode(sys_ol, "Lazo abierto: Bode")
    st.pyplot(fig_bode_ol)

with st.expander("Respuestas en el tiempo (lazo abierto)"):
    fig_time_ol = plot_time_responses(sys_ol, params_ol,
                                      "Lazo abierto: respuestas en el tiempo")
    st.pyplot(fig_time_ol)

st.markdown("---")

#Lazo cerrado

st.markdown("### Lazo cerrado (realimentación unitaria con $K_c$)")

k_cl = k + Kc
sys_cl = sig.TransferFunction([Kc], [m, c, k_cl])
params_cl = compute_second_order_params(m, c, k_cl)

col1, _ = st.columns([1, 1])

with col1:
    st.markdown("**Parámetros dinámicos (cerrado)**")
    z_cl = params_cl["zeta"]
    wn_cl = params_cl["wn"]
    wd_cl = params_cl["wd"]
    tp_cl = params_cl["tp"]
    tr_cl = params_cl["tr"]
    ts_cl = params_cl["ts"]

    st.write(f"ζcl = {z_cl:.4f} → {clasifica_zeta(z_cl)}")
    st.write(f"ωₙ,cl = {wn_cl:.4f} rad/s")
    if wd_cl is not None:
        st.write(f"ωd,cl = {wd_cl:.4f} rad/s")
    else:
        st.write("ωd,cl: no aplica (no hay oscilaciones).")
    st.write(f"ts,cl ≈ {ts_cl:.4f} s")
    if tp_cl is not None:
        st.write(f"tp,cl ≈ {tp_cl:.4f} s")
    else:
        st.write("tp,cl: no aplica (respuesta monótona).")
    if tr_cl is not None:
        st.write(f"tr,cl ≈ {tr_cl:.4f} s")
    else:
        st.write("tr,cl: se obtiene de la respuesta monótona.")

with st.expander("Polos y ceros (lazo cerrado)"):
    fig_pz_cl = plot_pzmap(sys_cl, "Lazo cerrado: polos y ceros")
    st.pyplot(fig_pz_cl)

with st.expander("Bode lazo cerrado"):
    fig_bode_cl = plot_bode(sys_cl, "Lazo cerrado: Bode")
    st.pyplot(fig_bode_cl)

with st.expander("Respuestas en el tiempo (lazo cerrado)"):
    fig_time_cl = plot_time_responses(sys_cl, params_cl,
                                      "Lazo cerrado: respuestas en el tiempo")
    st.pyplot(fig_time_cl)


Overwriting pages/2_⚙️_Masa_Resorte_Amortiguador.py


##Ejecutar el dashboard multipágina

In [10]:
# Descargar y configurar cloudflared para exponer el puerto 8501
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64
!mv cloudflared-linux-amd64 /usr/local/bin/cloudflared

# Ejecutar Streamlit
!streamlit run app.py &>/content/logs.txt &

# Exponer el puerto 8501 con Cloudflare Tunnel
!cloudflared tunnel --url http://localhost:8501 > /content/cloudflared.log 2>&1 &

# Leer la URL pública generada por Cloudflare
import time, re
time.sleep(5)

found_context = False
with open('/content/cloudflared.log') as f:
    for line in f:
        if "Your quick Tunnel has been created" in line:
            found_context = True
        if found_context:
            match = re.search(r'https?://\S+', line)
            if match:
                url = match.group(0)
                print(f'Tu aplicación está disponible en: {url}')
                break

import os
res = input("Digite (1) para finalizar la ejecución del Dashboard: ")

if res.upper() == "1":
    os.system("pkill streamlit")
    print("El proceso de Streamlit ha sido finalizado.")


Tu aplicación está disponible en: https://pushed-certified-packets-greensboro.trycloudflare.com
Digite (1) para finalizar la ejecución del Dashboard: 0
